In [1]:
### Don't mind about this 
import warnings
warnings.filterwarnings('ignore')
###

%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier # I will use the Sklearn to get kNN
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
import math
from statistics import mode

# Part A: 

### we need to consider that the features are Vocubulary.

In [2]:
Dict = pd.read_csv("vocabulary.txt", sep="\n")


In [3]:
feature_names = Dict.values.reshape((5821,1))
feature_names=feature_names.flatten()
feature_names = np.hstack((Dict.columns,feature_names))

In [4]:
baseball_train= pd.read_csv("baseball_train_set.csv", names=feature_names)
hockey_train= pd.read_csv("hockey_train_set.csv", names=feature_names)
hockey_test=  pd.read_csv("hockey_test_set.csv", names=feature_names)
baseball_test= pd.read_csv("baseball_test_set.csv", names=feature_names)

In [5]:
len(feature_names)

5822

In [6]:
print(baseball_train.shape)
print(hockey_train.shape)
print(hockey_test.shape)
print(baseball_test.shape)


(50, 5822)
(50, 5822)
(50, 5822)
(50, 5822)


In [7]:
baseball_train['Class_Label']=0 
baseball_test['Class_Label']=0
hockey_train['Class_Label']=1
hockey_test['Class_Label']=1

In [8]:
baseball_train.head()

'90  'em  'taters  0  00  000  00ecgillespie  01  011  013  ...  zeile's  \
0    0    0        0  0   0    0              1   0    0    0  ...        0   
1    0    0        0  0   0    0              0   0    0    0  ...        0   
2    0    0        0  0   0    0              0   0    0    0  ...        0   
3    0    0        0  1   0    0              0   0    0    0  ...        0   
4    0    0        0  0   0    0              0   0    0    0  ...        0   

   zelepukin  zero  zezel  zipper  zippety  zombo  zone  zubov  Class_Label  
0          0     0      0       0        0      0     0      0            0  
1          0     0      0       0        0      0     0      0            0  
2          0     0      0       0        0      0     0      0            0  
3          0     0      0       0        0      0     0      0            0  
4          0     0      0       0        0      0     0      0            0  

[5 rows x 5823 columns]

In [9]:
#Now we need to concatenate 

#Check this https://stackoverflow.com/questions/35528119/pandas-recalculate-index-after-a-concatenation
temp_train = [baseball_train,hockey_train]
train = pd.concat(temp_train)
temp_test = [baseball_test,hockey_test]
test = pd.concat(temp_test)
#Shuffle the data 
train = train.sample(frac=1).reset_index(drop=True)
test= test.sample(frac=1).reset_index(drop=True)

In [10]:
train.sum().sum()

17389

In [11]:
print(train['0'].sum()) #each word with frequencies

147


### Implementing the Naive Bayes

In [128]:
class NB:
    
    def __init__(self):
        self.Px_y= dict()
        self.Py= dict()
        self.cols= None
        
        
    def fit(self,X,y):
        
        # first calculate the probability of y equal each value
        labels,counts = np.unique(y,return_counts=True)

        m = y.shape[0]
        
        categoricalSum = {}
        
        
        for label in range(len(labels)):
            self.Py[labels[label]]=counts[labels[label]]/m
            #Check this: https://thispointer.com/pandas-sum-rows-in-dataframe-all-or-certain-rows/
            categoricalSum[labels[label]] = X[labels[label]==y].sum().sum() #will calculate each sum for 1s and 0s 
        
        
            
         
        self.cols = X.columns # Getting the feature names 

       
        print(categoricalSum)
        
        for col in X.columns: # loop each column
            self.Px_y[col] = {} #need dictinaory for each feature {0: probability, 1: probability} 
            feature = col
        
            for i in range(len(labels)):
                #This will give for each feature with respect to labels the probability.
                self.Px_y[feature][labels[i]] = np.log(((1 + X[feature][labels[i]==y].sum())/categoricalSum[labels[i]]))
                
            
                    
        return self
            

    def displayWordProbabilities(self):
        return self.Px_y
    
    def predict(self,x_test):
        
        # get P(Y | X_test)
        output = []
        test_instances = len(x_test)
        
        for instance in range(test_instances):
           
            out = []
            for label,prob in self.Py.items(): # loop over each category proba
                
                p=np.log(prob) #value of the label
                
                
                for i in range((len(X_test.iloc[instance]))): # loop over each feature in x_test
                    col = self.cols[i]
                    # We need to get the probabilty by summing the logs of the normal probability 
                    # then we will multiply with the frequency of each test word
                    p = (p + ((self.Px_y[col][label])*(x_test.iloc[instance][i])))
                    
                out.append((p,label))
            sor = sorted(out , key= lambda x: x[0], reverse= True) # to get the highest probability and take its label
            output.append(sor[0][1])
    
        return output
                    
                
        
        

In [129]:
X= train.iloc[:,:-1]
y= train['Class_Label']
X_test= test.iloc[:,:-1]
y_test= test['Class_Label']

In [130]:
X_test.shape

(100, 5822)

In [131]:
cle = NB()
cle.fit(X,y)
prediction = cle.predict(X_test)

{0: 7054, 1: 10285}


In [132]:
prediction

[0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1]

In [133]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80        50
           1       1.00      0.50      0.67        50

    accuracy                           0.75       100
   macro avg       0.83      0.75      0.73       100
weighted avg       0.83      0.75      0.73       100



### Displaying different probabilities to words:

In [135]:
show = cle.displayWordProbabilities()
print(show)

{"'90": {0: -8.861350110795998, 1: -9.238441802087058}, "'em": {0: -8.861350110795998, 1: -8.545294621527113}, "'taters": {0: -8.168202930236053, 1: -9.238441802087058}, '0': {0: -4.767005548573897, 1: -4.7498054323549175}, '00': {0: -3.9710009825742434, 1: -9.238441802087058}, '000': {0: -5.494054280809523, 1: -8.139829513418947}, '00ecgillespie': {0: -7.762737822127888, 1: -9.238441802087058}, '01': {0: -5.250432198151773, 1: -9.238441802087058}, '011': {0: -8.861350110795998, 1: -8.545294621527113}, '013': {0: -8.861350110795998, 1: -8.545294621527113}, '014': {0: -8.861350110795998, 1: -8.139829513418947}, '021': {0: -8.861350110795998, 1: -8.545294621527113}, '037': {0: -8.861350110795998, 1: -8.545294621527113}, '038': {0: -8.861350110795998, 1: -8.545294621527113}, '050': {0: -8.861350110795998, 1: -8.139829513418947}, '051': {0: -8.861350110795998, 1: -8.545294621527113}, '053': {0: -8.861350110795998, 1: -8.545294621527113}, '055': {0: -8.861350110795998, 1: -8.545294621527113

In [182]:
#With the help of https://stackoverflow.com/questions/62096337/how-do-you-find-which-words-a-trained-naive-bayes-classifier-uses-to-make-decisi
#Actual Probability without log 
def gettingProbability(logValues):
    probabilities = np.exp(logValues)
    return probabilities



In [207]:
data_list = list(show.values())
df = pd.DataFrame(data_list)
gettingProbability(df)

0         1
0     0.000142  0.000097
1     0.000142  0.000194
2     0.000284  0.000097
3     0.008506  0.008653
4     0.018855  0.000097
...        ...       ...
5817  0.000142  0.000097
5818  0.000142  0.000097
5819  0.000142  0.000097
5820  0.000142  0.000194
5821  0.000142  0.000194

[5822 rows x 2 columns]

## B. Using Sklearn to train naivebayes

In [183]:
from sklearn.naive_bayes import MultinomialNB

naiveBayesClassifier = MultinomialNB()
naiveBayesClassifier.fit(X, y)
predictionNBClassifier = naiveBayesClassifier.predict(X_test)
predictionNBClassifier

array([0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1])

In [184]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, predictionNBClassifier))

              precision    recall  f1-score   support

           0       0.94      0.62      0.75        50
           1       0.72      0.96      0.82        50

    accuracy                           0.79       100
   macro avg       0.83      0.79      0.78       100
weighted avg       0.83      0.79      0.78       100



### Getting the probability of each word in each class

In [187]:
x=gettingProbability(naiveBayesClassifier.feature_log_prob_.T)

In [211]:
pd.DataFrame(x,columns = ['baseball','hockey'])

baseball    hockey
0     0.000078  0.000062
1     0.000078  0.000124
2     0.000155  0.000062
3     0.004660  0.005526
4     0.010329  0.000062
...        ...       ...
5817  0.000078  0.000062
5818  0.000078  0.000062
5819  0.000078  0.000062
5820  0.000078  0.000124
5821  0.000078  0.000124

[5822 rows x 2 columns]

## C: Using TF-IDF to preprocess the data instead

In [214]:
X_TFIDF = X.copy()
y_TFIDF = y.copy()
X_test_TFIDF = X_test.copy()
y_test_TFIDF = y_test.copy()

### Keep in mind that I took the TF-IDF from my project as I implemented from scratch once

In [221]:
import math
def convertToTFIDF(X_TFIDF):
    freq_dict={}
    for col in X_TFIDF.columns:
        freq_dict[col]=0

    for i in range(len(X)):
        for col in X_TFIDF.columns:
            if X_TFIDF[col].iloc[i]!=0:
                freq_dict[col]+=1
    IDF = {}
    doc_length = X_TFIDF.shape[0]
    for w in X_TFIDF.columns: 
        IDF[w] = math.log(doc_length/(freq_dict[w]+1))


In [222]:
def conversion_(X_TFIDF):
    for col in X.columns: 
        X_TFIDF[col]=X_TFIDF[col].apply(lambda x: x*IDF[col]) 

In [223]:
conversion_(X_TFIDF)
conversion_(X_test_TFIDF)

# Now Train with A and B 

In [228]:
#My Own Implementation
cle_TFIDF = NB()



In [229]:
cle_TFIDF.fit(X_TFIDF,y_TFIDF)

{0: 42541.017036150966, 1: 60778.37081284498}


In [231]:
prediction_TFIDF = cle_TFIDF.predict(X_test_TFIDF)
prediction_TFIDF

[0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1]

In [232]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test_TFIDF, prediction_TFIDF))

              precision    recall  f1-score   support

           0       0.87      0.94      0.90        50
           1       0.93      0.86      0.90        50

    accuracy                           0.90       100
   macro avg       0.90      0.90      0.90       100
weighted avg       0.90      0.90      0.90       100



In [225]:
#using Sklearn: 

from sklearn.naive_bayes import MultinomialNB

naiveBayesClassifier_TFIDF = MultinomialNB()
naiveBayesClassifier_TFIDF.fit(X_TFIDF, y_TFIDF)
predictionNBClassifier_TFIDF = naiveBayesClassifier_TFIDF.predict(X_test_TFIDF)
predictionNBClassifier_TFIDF


array([0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1])

In [226]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test_TFIDF, predictionNBClassifier_TFIDF))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90        50
           1       0.92      0.88      0.90        50

    accuracy                           0.90       100
   macro avg       0.90      0.90      0.90       100
weighted avg       0.90      0.90      0.90       100



In [227]:
#Showing the probability for each word

x_TFIDF=gettingProbability(naiveBayesClassifier_TFIDF.feature_log_prob_.T)

pd.DataFrame(x_TFIDF,columns = ['baseball','hockey'])

baseball    hockey
0     0.000021  0.000015
1     0.000021  0.000245
2     0.000337  0.000015
3     0.007094  0.007676
4     0.041791  0.000015
...        ...       ...
5817  0.000021  0.000015
5818  0.000021  0.000015
5819  0.000021  0.000015
5820  0.000021  0.000245
5821  0.000021  0.000245

[5822 rows x 2 columns]